# # Projet : Planifier mes prochaines vacances 

### Utilisons les API et le web-scraping pour déterminer quels seraient les plus beaux endroits à visiter pour des vacances de dernière minute la semaine prochaine !

## Selon <a href="https://one-week-in.com/35-cities-to-visit-in-france/" target="_blank">ce site web</a>, voici les 35 meilleurs endroits à visiter en France en 2020 :

 ```python
["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]
```

### Utilisez <a href="https://nominatim.org/" target="_blank">https://nominatim.org/</a> pour obtenir les coordonnées gps de toutes les villes (aucun abonnement requis).

### Documentation : <a href="https://nominatim.org/release-docs/develop/api/Search/" target="_blank">https://nominatim.org/release-docs/develop/api/Search/</a>

### 3. Déterminez la liste des villes où le temps sera le plus agréable dans les 7 prochains jours.

Par exemple, vous pouvez utiliser les valeurs de `daily.pop` et `daily.rain` pour calculer le volume de pluie attendu dans les 7 prochains jours... Mais ce n'est qu'un exemple, en fait vous pouvez avoir des opinions différentes sur ce que serait un "beau temps" 😎. Peut-être que le critère le plus important pour vous est la température ou l'humidité, alors n'hésitez pas à changer les règles !

### 4. Enregistrez tous les résultats dans un fichier .csv, vous l'utiliserez plus tard 😉 .

Vous pouvez enregistrer toutes les informations qui vous semblent importantes ! N'oubliez pas de sauvegarder le nom des villes, et aussi de créer une colonne contenant un identifiant unique (id) de chaque ville (c'est important pour la suite du projet).

### 5. Utilisez plotly pour afficher les meilleures destinations sur une carte.

# Import des librairies

In [1]:
import json
import requests
import pandas as pd
import plotly.express as px
from plotly import graph_objs as go

In [3]:
!pip install plotly
!pip install boto3
!pip install python-box
!pip install pyyaml
#!jupyter labextension install jupyterlab-plotly@4.8.1

In [4]:
city_names = [
    "Mont Saint Michel",
    "St Malo",
    "Bayeux",
    "Le Havre",
    "Rouen",
    "Paris",
    "Amiens",
    "Lille",
    "Strasbourg",
    "Chateau du Haut Koenigsbourg",
    "Colmar",
    "Eguisheim",
    "Besancon",
    "Dijon",
    "Annecy",
    "Grenoble",
    "Lyon",
    "Gorges du Verdon",
    "Bormes les Mimosas",
    "Cassis",
    "Marseille",
    "Aix en Provence",
    "Avignon",
    "Uzes",
    "Nimes",
    "Aigues Mortes",
    "Saintes Maries de la mer",
    "Collioure",
    "Carcassonne",
    "Foix",
    "Toulouse",
    "Montauban",
    "Biarritz",
    "Bayonne",
    "La Rochelle"
]

In [5]:
dataset = pd.DataFrame(columns=['city_id' , 'name', 'latitude', 'longitude', 'main_weather', 'expected_rain', 'day_temperature'])

In [7]:
#  IDENTIFIANT D'APPLICATION (VOUS DEVEZ VOUS ABONNER À OPENWEATHERMAP)

API_KEY = "755cc401942e8750f3ee4a975b76fd81"

In [8]:
for i in range(len(city_names)):
    
    print("Making requests for {}".format(city_names[i]))
    
  
    # Utiliser l'api nominatim pour obtenir les coordonnées GPS de la ville
    
    NOMINATIM_URL = 'https://nominatim.openstreetmap.org/search'
    params = {
        "city": city_names[i],
        "country": "France",
        "format": "json"
    }
    
    r = requests.get(NOMINATIM_URL, params=params)
    res = r.json()
    
    # ex: Gorges du Verdon
    if len(res) < 1:
        params = {
            "q": city_names[i],
            "country": "France",
            "format": "json"
        }

        r = requests.get(NOMINATIM_URL, params=params)
        res = r.json()    
    
    dataset.loc[i,'city_id'] = i
    dataset.loc[i,'name'] = city_names[i]
    dataset.loc[i, 'latitude'] = res[0]['lat']
    dataset.loc[i, 'longitude'] = res[0]['lon']
    
  
    # Utiliser l'api openweathermap pour obtenir le temps pour les 7 prochains jours
    
    params = {
        "lat": dataset.loc[i, 'latitude'],
        "lon": dataset.loc[i, 'longitude'],
        "exclude": "current,minutely,hourly",
        "units": "metric",
        "appid": API_KEY,
    }

    r = requests.get('https://api.openweathermap.org/data/2.5/onecall', params=params)
    res = r.json()
    
   
    # Calculer le volume attendu de pluie
    
    expected_rain = 0
    # print(res)
    for d in res['daily']:
        if 'rain' in d.keys():
            expected_rain += d['pop'] * d['rain']
            
 
    # Calculer la température moyenne du jour
    
    temperatures = pd.Series([d['temp']['day'] for d in res['daily']])
    mean_temperature = temperatures.mean()


    # Extrait du temps le plus probable
    
    weathers = pd.Series([d['weather'][0]['main'] for d in res['daily']])
    main_weather = weathers.mode()[0]
    
    dataset.loc[i,'main_weather'] = main_weather
    dataset.loc[i,'expected_rain'] = expected_rain
    dataset.loc[i,'day_temperature'] = mean_temperature


Making requests for Mont Saint Michel
Making requests for St Malo
Making requests for Bayeux
Making requests for Le Havre
Making requests for Rouen
Making requests for Paris
Making requests for Amiens
Making requests for Lille
Making requests for Strasbourg
Making requests for Chateau du Haut Koenigsbourg
Making requests for Colmar
Making requests for Eguisheim
Making requests for Besancon
Making requests for Dijon
Making requests for Annecy
Making requests for Grenoble
Making requests for Lyon
Making requests for Gorges du Verdon
Making requests for Bormes les Mimosas
Making requests for Cassis
Making requests for Marseille
Making requests for Aix en Provence
Making requests for Avignon
Making requests for Uzes
Making requests for Nimes
Making requests for Aigues Mortes
Making requests for Saintes Maries de la mer
Making requests for Collioure
Making requests for Carcassonne
Making requests for Foix
Making requests for Toulouse
Making requests for Montauban
Making requests for Biarrit

In [ ]:
 len(city_names)

# Mettre les villes récupérées dans un cadre de données

In [9]:
dataset = dataset.astype({'city_id': int,
                          'latitude': float,
                          'longitude': float,
                          'expected_rain': float,
                          'day_temperature': float,
                         })

dataset.loc[:, 'rank'] = dataset['expected_rain'].rank(method='min').astype(int)
dataset.loc[:, 'inverted_rank'] = dataset['expected_rain'].rank(method='min', ascending=False).astype(int)

dataset = dataset.sort_values(by=['expected_rain', 'day_temperature'], ascending=[True, False]).reset_index(drop=True)
display(dataset)

,city_id,name,latitude,longitude,main_weather,expected_rain,day_temperature,rank,inverted_rank
0,21,Aix en Provence,43.529842,5.447474,Clouds,0.2327,14.34500,1,35
1,22,Avignon,43.949249,4.805901,Rain,2.2042,15.93750,2,34
2,8,Strasbourg,48.584614,7.750713,Clouds,2.5916,12.34875,3,33
3,2,Bayeux,49.276462,-0.702474,Rain,2.8621,11.71000,4,32
4,20,Marseille,43.296174,5.369953,Clouds,3.5174,14.50125,5,31
5,10,Colmar,48.077752,7.357964,Clouds,3.7312,12.50125,6,30
6,11,Eguisheim,48.044797,7.307962,Clouds,4.0167,12.54000,7,29
7,3,Le Havre,49.493898,0.107973,Clouds,4.0682,11.20625,8,28
8,9,Chateau du Haut Koenigsbourg,48.249523,7.345492,Rain,4.0972,9.56375,9,27
9,17,Gorges du Verdon,43.749656,6.328562,Rain,4.5448,10.96375,10,26


# Les meilleurs endroits pour un voyage la semaine prochaine

In [13]:
print('Les meilleurs endroits pour un voyage la semaine prochaine sont : ')
print()

for i, row in dataset.loc[dataset['rank']==1,:].iterrows():
    print("{} -- Mostly {} with temperature {} °C".format(row['name'], row['main_weather'], row['day_temperature']))

Les meilleurs endroits pour un voyage la semaine prochaine sont : 

Aix en Provence -- Mostly Clouds with temperature 14.345000000000002 °C


# Enregistrement du fichier 

In [14]:
dataset.to_csv('res/1_destinations.csv', index=False)

# Visualisation sur une carte destinations

In [15]:
fig = px.scatter_mapbox(dataset, lat="latitude", lon="longitude", hover_name = 'name', zoom = 4,
                        hover_data = ['main_weather', 'expected_rain', 'day_temperature'], 
                        color = 'day_temperature', color_continuous_scale = 'Bluered', size = 'inverted_rank',
                        mapbox_style="carto-positron")
fig.show('iframe')

In [16]:
dataset.to_csv('res/1_destinations.csv', index=False)